# What do you need to know
- It's not my original. I'm just learn the Blockchain by following this [tutorial](https://hackernoon.com/learn-blockchains-by-building-one-117428612f46)
- I did *NOT* fully understand how it works. (sign..)
- This tutorial requires these:

| what you need  | my thoughts | 
| --- | --- | 
| Python 3.6.* | I think it could be easily transformed into Python 2 code. | 
| flask, requests | Just ```pip install```! |

<hr>

## Step1 - Build a Blockchain

#### create a blockchain class 

In [9]:
import hashlib 
import json 
from time import time

In [10]:
class Blockchain(object):
    
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        
        # create the genesis Block 
        self.new_block(previous_hash=1, proof=100)
        
        
    def new_block(self, proof, previous_hash=None):
        """
        Creates a new Block in the Blockchain
        
        proof          int   the proof given by the Proof-of-Work algorithm 
        previous_hash  str   hash of previous Block (optional-param)
        return         dict  new Block
        """
        
        block = {                             # add block info 
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1])
        }
        
        # reset the current list of transactions 
        self.current_transactions = []        # all transactions?? (not sure)
        
        self.chain.append(block)              # block-Chain (append block-info)
        return block                          # return block-info
        
    
    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block
        
        sender         str   address of the sender
        recipient      str   address of the recipient
        amount         int   amount 
        return         int   the index of the Block that'll hold this transaction
        """
        
        self.current_transactions.append({    # add transac info (as dict)
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })
        
        return self.last_block['index'] + 1   # index thing :)
       
    
    @property
    def last_block(self):
        return self.chain[-1]                 # last block (.. last index)
    
    
    @staticmethod
    def hash(block):
        """
        Creates a SHA-256 hash of a Block
        
        block           dict  Block 
        return          str 
        """ 
        
        # we must make sure that 
        #   the Dict is ordered, or we'll have inconsistent hashes 
        block_string = json.dumps(block, sort_keys=True).encode()
        
        return hashlib.sha256(block_string).hexdigest()
    
    
    # Yes! We're going to implementing basic Proof-of-Work!
    
    
    def proof_of_work(self, last_proof):
        """
        Simple Proof-of-Work algorithm:
        --- Find a num p' such that hash(pp') contains leading 4 zeros,
        --- p is the previous proof, and p' is the new proof.
        
        last_proof      int 
        return          int 
        """
        
        proof = 0
        
        while self.valid_proof(last_proof, proof) is False:
            proof = proof + 1 
        
        return proof 
    
    
    @staticmethod 
    def valid_proof(last_proof, proof):
        """
        Validates the Proof: Does hash(last_proof, proof) contains 4 leading zeros?
        
        last_proof       int   previous Proof 
        proof            int   current Proof 
        return           bool  True if correct, False if not. (XD)
        """
        
        guess      = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        
        return guess_hash[:4] == "0000"
    
    
    # Yes! Yes! We're going to implementing the 'Consensus' shit
    
    def register_node(self, address):
        """
        Add a new node to the list of nodes
        
        address           str    address of node    E.g. 'http://192.168.0.5:5000'
        return            None 
        """
    
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)
        
    def valid_chain(self, chain):
        """
        Determine if a given blockchain is valid {
            looping through each block 
            and verifying both the hash and the proof 
        }
        
        chain              list   a block chain 
        return             bool   True if valid, False if not (XDD)
        """
        
        last_block = chain[0]
        current_index = 1 
        
        while current_index < len(chain):
            
            block = chain[current_index]
            
            print(f'{last_block}')
            print(f'{block}')
            print('\n-----------\n')
            
            # check that the hash of the Block is correct 
            if block['previous_hash'] != self.hash(last_block):
                return False 
            
            last_block = block
            current_index = current_index + 1 
            
        return True
    
    
    def resolve_conflicts(self):
        """
        This is our Consensus Algorithm, it resolves conflicts 
        by replacing our chain with the longest one in the network.
        
        return      bool        True if our chain was replaced, False if not.
        """
        
        neighbors = self.nodes
        new_chain = None 
        
        # we're only looking for chains longer than ours 
        max_length = len(self.chain)
        
        # Grab and verify the chains from all the nodes in our network 
        for node in neighbors:
            response = requests.get(f'http://{node}/chain')
            
            if response.status_code == 200:
                length = response.json()['length']
                chain  = response.json()['chain']
                
                # Check if the length is longer and the chain is valid 
                if length > max_length and self.valid_chain(chain):
                    max_length = length
                    new_chain  = chain 
              
        # Replace our chain if we discovered a new, valid chain longer than ours
        if new_chain:
            self.chain = new_chain 
            return True 
        
        return False 

#### about Proof-of-Work

In [11]:
basis = """
    A Proof-of-Work algorithm is how new Blocks are created/mined on the Blockchain.
    
    The goal of PoW is to discover a number which solves a problem.
        The num must be 'difficult to find but easy to verify'.
        Computationally speaking, by anyone on the network.
    
    This is the core idea behind the Proof-of-Work.
"""

# Oh! Btw, the f'xxx' is a new feature in Python 3.6
#   here's the pep link: 
#     docs.python.org/3/whatsnew/3.6.html#pep-498-formatted-string-literals

import hashlib 

x = 5  # nothing particular
y = 0  # sort of index ( it denotes 'at XXth time, we got a str ends with a num we specified' )

# f'{x*y}'    --  num 
# .encode     --  encode num for sha256 (  either b'' or ''.encode()  )
# .hexdigest  --  human-readable encoded string 
# [-1]        --  pick the last char in the '.hexdigest()' result 
# != "0"      --  nothing particular either (just give a condition to stop the loop)

while hashlib.sha256( f'{x*y}'.encode() ).hexdigest()[-1] != "3":
    print((x,y),end='\t')
    y = y + 1 

# values of y and correspond hash 
'The solution is y = {}'.format(y)
'The hash is {}'.format(hashlib.sha256(str(x*y).encode()).hexdigest())
           

(5, 0)	(5, 1)	(5, 2)	(5, 3)	(5, 4)	(5, 5)	(5, 6)	

'The solution is y = 7'

'The hash is 9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739e5ff3827ffb3634953'

## Step 2 - Our Blockchain as an API

#### setting up Flask

In [12]:
we_will_create_three_methods = """
    /transactions/new    create a new transaction to a block 
    /mine                to tell our server to mine a new block 
    /chain               to return the full Blockchain  
"""

from textwrap import dedent 
from time import time 
from uuid import uuid4 

from flask import Flask, jsonify, request

In [13]:
# Instantiate our Node 
app = Flask(__name__)

# Generate a globally unique addr for this node 
node_identifier = str(uuid4()).replace('-','')

# Instantiate the Blockchain 
blockchain = Blockchain()

#### routes!

In [14]:
@app.route('/mine',methods=['GET'])
def mine():
    # We run the Proof-of-Work algorithm to get the next proof ...
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)
    
    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0",
        recipient=node_identifier,
        amount=1,
    )    
    
    # Forge the next Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block         = blockchain.new_block(proof, previous_hash)
    
    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash']
    }
    
    return jsonify(response), 200
    

@app.route('/transactions/new',methods=['POST'])
def new_transaction():
    values = request.get_json()
    
    # Check that the required fileds are in the 'POST'ed data 
    required = ['sender', 'recipient', 'amount']
    
    if not all(k in values for k in required):
        return 'Missing values', 400
    
    # Create a new Transaction 
    index = blockchain.new_transaction(
        values['sender'], 
        values['recipient'],
        values['amount'],
    )
    
    response = {'message': f'Transaction will be added to Block {index}'}
    
    return jsonify(response), 201

    
@app.route('/chain',methods=['GET'])
def full_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain)
    }
    return jsonify(response), 200


@app.route('/nodes/register', methods=['POST'])
def register_nodes():
    values = request.get_json()
    
    nodes = values.get('nodes')
    if nodes is None:
        return "Error: Please supply a valid list of nodes", 400
    
    for node in nodes:
        blockchain.register_node(node)
        
    response = {
        'message': 'New nodes have been added',
        'total_nodes': list(blockchain.nodes),
    }
    
    return jsonify(response), 201


@app.route('/nodes/resolve', methods=['GET'])
def consesus():
    replaced = blockchain.resolve_conflicts()
    
    if replaced:
        response = {
            'message': 'Our chain was replaced',
            'new_chain': blockchain.chain
        }
    else:
        response = {
            'message': 'Our chain is authoritative',
            'chain': blockchain.chain
        }
        
    return jsonify(response), 200

## Step 3 - Interacting with our Blockchain

#### The rest of them is optional <small>(sort of)</small>.

In [15]:
#  urls: 
#    [ /mine, /chain ]

# app.run(host='0.0.0.0',port=5000)

In [16]:
# then we're gonna use a 'Postman' ( or not )

let_us_try = '''
    mining a block            making a 'GET' request to ../mine
    create a transaction      making a 'POST' request (Nope, u have to use 'curl')
    inspect the full chain    access ../chain
'''

the_curl_cmd = ''' 
    curl -X POST -H "Content-Type: application/json" -d '{
     "sender": "d4ee26eee15148ee92c6cd394edd974e",
     "recipient": "someone-other-address",
     "amount": 5
    }' "http://localhost:5000/transactions/new 
'''

